In the other jupyter notebook I created for myself a demographic data shapefile at the block level, which is stored in `C:/Users/madie/OneDrive/data/ipums/VA_block_2010_pop.zip`. What I need to now do is aggregate that block-level data into the precinct shapefiles that I got from the VEST team's dataverse. 

In [1]:
import geopandas
import pandas as pd
import maup

# turn off annoying geoseries isna warnings
import warnings
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)

blocks = geopandas.read_file("zip://C:/Users/madie/OneDrive/data/ipums/VA_block_2010_pop.zip")
precincts_15 = geopandas.read_file("zip://C:/Users/madie/OneDrive/data/vest/va_2015_statehouse.zip")
#precincts_17 = geopandas.read_file("zip://C:/Users/madie/OneDrive/data/vest/va_2017_statehouse.zip")
#precincts_19 = geopandas.read_file("zip://C:/Users/madie/OneDrive/data/vest/va_2019_statehouse.zip")

In [2]:
blocks.crs

<Projected CRS: ESRI:102003>
Name: USA_Contiguous_Albers_Equal_Area_Conic
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [3]:
precincts_15.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [4]:
# change crs of blocks so that it's projected
blocks = blocks.to_crs(epsg=2283)
blocks.crs

<Projected CRS: EPSG:2283>
Name: NAD83 / Virginia North (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - Virginia - counties of Arlington; Augusta; Bath; Caroline; Clarke; Culpeper; Fairfax; Fauquier; Frederick; Greene; Highland; King George; Loudoun; Madison; Orange; Page; Prince William; Rappahannock; Rockingham; Shenandoah; Spotsylvania; Stafford; Warren; Westmoreland.
- bounds: (-80.06, 37.77, -76.51, 39.46)
Coordinate Operation:
- name: SPCS83 Virginia North zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [5]:
# change crs of blocks so that it's projected
precincts_15 = precincts_15.to_crs(epsg=2283)
precincts_15.crs

<Projected CRS: EPSG:2283>
Name: NAD83 / Virginia North (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - Virginia - counties of Arlington; Augusta; Bath; Caroline; Clarke; Culpeper; Fairfax; Fauquier; Frederick; Greene; Highland; King George; Loudoun; Madison; Orange; Page; Prince William; Rappahannock; Rockingham; Shenandoah; Spotsylvania; Stafford; Warren; Westmoreland.
- bounds: (-80.06, 37.77, -76.51, 39.46)
Coordinate Operation:
- name: SPCS83 Virginia North zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
# remove any bowties (little imperfections in the polygons)
blocks.geometry = blocks.buffer(0)

In [7]:
blocks.columns

Index(['GEOID10', 'GISJOIN', 'NAME10', 'pop', 'vap', 'TotPop', 'BlackPop',
       'HispPop', 'VAP_1', 'BlackVAP', 'HispVAP', 'geometry'],
      dtype='object')

In [9]:
precincts_15.geometry = precincts_15.buffer(0)

In [11]:
precincts_15.columns

Index(['COUNTYFP', 'LOCALITY', 'VTDST', 'PRECINCT', 'HOD_DIST', 'G15HODDEM',
       'G15HODREP', 'G15HODIND', 'G15HODLIB', 'G15HODGRE', 'G15HODWRI',
       'geometry'],
      dtype='object')

In [12]:
variables = ['pop', 'vap', 'TotPop', 'BlackPop', 'HispPop', 'VAP_1', 'BlackVAP', 'HispVAP']

with maup.progress():
    assignment = maup.assign(blocks, precincts_15)
precincts_15[variables] = blocks[variables].groupby(assignment).sum()
precincts_15.columns

100%|██████████████████████████████████████████████████████████████████████████████| 2577/2577 [03:54<00:00, 11.00it/s]


Index(['COUNTYFP', 'LOCALITY', 'VTDST', 'PRECINCT', 'HOD_DIST', 'G15HODDEM',
       'G15HODREP', 'G15HODIND', 'G15HODLIB', 'G15HODGRE', 'G15HODWRI',
       'geometry', 'pop', 'vap', 'TotPop', 'BlackPop', 'HispPop', 'VAP_1',
       'BlackVAP', 'HispVAP'],
      dtype='object')

In [13]:
precincts_15.head()

,COUNTYFP,LOCALITY,VTDST,PRECINCT,HOD_DIST,G15HODDEM,G15HODREP,G15HODIND,G15HODLIB,G15HODGRE,G15HODWRI,geometry,pop,vap,TotPop,BlackPop,HispPop,VAP_1,BlackVAP,HispVAP
0,003,Albemarle County,000202,University Hall,057,221,0,0,0,0,5,"POLYGON Z ((11474704.102 6692814.045 0.000, 11...",7704.0,7442.0,7704.0,572.0,425.0,7442.0,542.0,405.0
1,001,Accomack County,000101,Chincoteague,100,178,703,0,0,0,2,"POLYGON Z ((12370063.644 6661341.644 0.000, 12...",2941.0,2487.0,2941.0,21.0,49.0,2487.0,17.0,31.0
2,001,Accomack County,000201,Atlantic,100,67,339,0,0,0,0,"POLYGON Z ((12319939.723 6651354.425 0.000, 12...",1595.0,1278.0,1595.0,266.0,81.0,1278.0,210.0,53.0
3,001,Accomack County,000202,Greenbackville,100,97,306,0,0,0,0,"POLYGON Z ((12348275.656 6673222.628 0.000, 12...",2040.0,1700.0,2040.0,133.0,48.0,1700.0,105.0,29.0
4,001,Accomack County,000301,Oak Hall,100,221,269,0,0,0,0,"POLYGON Z ((12304918.630 6669238.329 0.000, 12...",3553.0,2765.0,3553.0,1812.0,99.0,2765.0,1379.0,61.0
